In [2]:
import yfinance as yf 
import pandas as pd 
from datetime import datetime, timedelta 
import numpy as np 
from scipy.optimize import minimize
import time

tickers = ['BND','AMZN','MSFT']

end_date = datetime.today()
start_date = end_date - timedelta(days = 5*365)
time.sleep(2)

adj_close_df = pd.DataFrame()

for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)
    adj_close_df[ticker] = data['Close']

print(adj_close_df)
    

log_returns = np.log(adj_close_df / adj_close_df.shift(1))
log_returns = log_returns.dropna()

cov_matrix = log_returns.cov()*252
print(cov_matrix)


def standard_deviation (weights, cov_matrix):
    variance = weights.T @ cov_matrix @ weights
    return np.sqrt(variance)

def expected_return (weights, log_returns):
    return np.sum(log_returns.mean()*weights)*252

def sharp_ratio (weights, log_returns, cov_matrix, risk_free_rate):
    return(expected_return(weights, log_returns)- risk_free_rate) / standard_deviation(weights, cov_matrix)
   

risk_free_rate = 0.02
def neg_sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return -sharp_ratio(weights, log_returns, cov_matrix, risk_free_rate)

constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
bounds = [(0, 0.5) for _ in range(len(tickers))]

initial_weights = np.array([1/len(tickers)]*len(tickers))
print(initial_weights)


optimized_results = minimize(neg_sharpe_ratio, initial_weights, args=(log_returns, cov_matrix, risk_free_rate), method='SLSQP', constraints=constraints, bounds=bounds)

optimal_weights = optimized_results.x

print("optimal weights:")
for ticker, weight in zip(tickers, optimal_weights):
    print(f"{ticker}: {weight:.4f}")


print()
optimal_portfolio_return = expected_return(optimal_weights, log_returns)
optimal_portfolio_volatility = standard_deviation(optimal_weights, cov_matrix)
optimal_sharp_ratio = sharp_ratio(optimal_weights, log_returns, cov_matrix, risk_free_rate)

print(f"Expected Annual Returns: {optimal_portfolio_return:.4f}")
print(f"Expected volatility: {optimal_portfolio_volatility:.4f}")
print(f"Sharp Ratio: {optimal_sharp_ratio:.4f}")






     
    


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['BND']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AMZN']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['MSFT']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Empty DataFrame
Columns: [BND, AMZN, MSFT]
Index: []
      BND  AMZN  MSFT
BND   NaN   NaN   NaN
AMZN  NaN   NaN   NaN
MSFT  NaN   NaN   NaN
[0.33333333 0.33333333 0.33333333]
optimal weights:
BND: 0.3333
AMZN: 0.3333
MSFT: 0.3333

Expected Annual Returns: 0.0000
Expected volatility: nan
Sharp Ratio: nan


c:\Users\asus\AppData\Local\Programs\Python\Python38-32\lib\site-packages\numpy\lib\function_base.py:495: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
c:\Users\asus\AppData\Local\Programs\Python\Python38-32\lib\site-packages\numpy\core\_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
c:\Users\asus\AppData\Local\Programs\Python\Python38-32\lib\site-packages\pandas\core\frame.py:10217: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
c:\Users\asus\AppData\Local\Programs\Python\Python38-32\lib\site-packages\numpy\lib\function_base.py:2680: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
c:\Users\asus\AppData\Local\Programs\Python\Python38-32\lib\site-packages\numpy\lib\function_base.py:2680: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
